#Installing and importing the required libraries

In [ ]:
!pip install javalang

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install anytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 405 kB/s 


In [ ]:
import javalang
import re

In [ ]:
'''
text = """class PalindromeExample{  
 public static void main(String args[]){  
  int r,sum=0,temp;    
  int n=454;//It is the number variable to be checked for palindrome  
  
  temp=n;    
  while(n>0){    
   r=n%10;  //getting remainder  
   sum=(sum*10)+r;    
   n=n/10;    
  }    
  if(temp==sum)    
   System.out.println("palindrome number ");    
  else    
   System.out.println("not palindrome");    
}}"""
'''

#Generating AST by reading the filename

In [ ]:
#tokens = javalang.tokenizer.tokenize(text)
def get_ast(filename):
  f = open(filename, "r")
  text = f.read()
  out = re.sub('//.*?\n|/\*.*?\*/', '', text, flags=re.S)
  #print(out)
  tree = javalang.parse.parse(out)
  return tree
#parser.parse_expression()

In [ ]:
#print(tree)

In [ ]:
'''
for path, node in tree:
    print("--------------------------------")
    print("NODE: ",node.__class__.__name__)#.value)
    cnt=0
    for i in path:
      print("=======")
      print(i)
      cnt+=1
    #print("+++++++++++++++++++++")
    print(cnt)
    print("Children: ",node.children)
'''

'\nfor path, node in tree:\n    print("--------------------------------")\n    print("NODE: ",node.__class__.__name__)#.value)\n    cnt=0\n    for i in path:\n      print("=======")\n      print(i)\n      cnt+=1\n    #print("+++++++++++++++++++++")\n    print(cnt)\n    print("Children: ",node.children)\n'

In [ ]:
#javalang.ast.walk_tree(tree)

<generator object walk_tree at 0x7f192d27d450>

#Getting the token of each node

In [ ]:
def get_token(root):
  token=""
  if type(root)==str:
    token=root
    return token
  else:
    token = root.__class__.__name__
    return token

#Getting the child nodes of each parent node

In [ ]:
from javalang.ast import Node
def get_children(root):
    if root.__class__.__name__=='CompilationUnit':
        #print(root)
        children = root.children
    #elif root.__class__.__name__=='ReferenceType':
        #print(root)
    #    children = root.children
    #elif ((root==None) or (type(root)==list and (len(root)==0 or root[0]==None))):
    #  children=[]
    elif isinstance(root, set):
        children = list(root)
    #elif type(root)==str:
    #    children = list(root)
    elif isinstance(root, javalang.ast.Node):
         children = root.children
    else:
        #print(root)
        children = []
    #print(children)
    
    def expand(nested_list):
        for item in nested_list:
            if isinstance(item, list):
                for sub_item in expand(item):
                    yield sub_item
            elif item:
                yield item
    #print(list(expand(children)))
    return list(expand(children))
    
    #return children


#Calling both the above functions to collect all the nodes into a list 

In [ ]:
import random
def get_trees(current_node, parent_node, order,nodes):
    #print(current_node)
    #fields = list(ast.iter_fields(current_node))
    #print(fields)
    token, children = get_token(current_node), get_children(current_node)
    node = Node([order,token], parent=parent_node, order=order)
    #print(parent_node)
    #if(parent_node.name=='')
    p = str(parent_node.name[0])+"--"+str(parent_node.name[1])
    c = str(order)+"--"+str(token)
    nodes.append((p,c))
    #print(nodes)
    #print(children)
    for child_order in range(len(children)):
        #print(order)
        get_trees(children[child_order], node, order+str(random.randint(1,100000)+1),nodes)

In [ ]:
from anytree import Node

def get_java_nodes(filename):
  tree = get_ast(filename)
  # Initialize head node of the code.
  
  head = Node(["1",tree.__class__.__name__])
  # Recursively construct AST tree.
  nodes = []
  #nodes.append((None,str(head.name[1])+str(head.name[0])))
  #print(get_children(tree))
  for child_order in range(len(get_children(tree))):
      #print(child_order)
      #print(get_children(tree)[child_order])
      get_trees(get_children(tree)[child_order], head,"1"+str(int(child_order)+1), nodes)

  return nodes

In [ ]:
filename='/content/ON3_8.java'
nodes=get_java_nodes(filename)

In [ ]:
nodes

[('0--CompilationUnit', '11--ClassDeclaration'),
 ('11--ClassDeclaration', '1163223--set'),
 ('1163223--set', '116322317311--public'),
 ('11--ClassDeclaration', '113848--ON3_8'),
 ('11--ClassDeclaration', '1182762--MethodDeclaration'),
 ('1182762--MethodDeclaration', '118276243846--set'),
 ('118276243846--set', '11827624384630314--public'),
 ('118276243846--set', '11827624384619111--static'),
 ('1182762--MethodDeclaration', '118276229915--BasicType'),
 ('118276229915--BasicType', '11827622991598599--int'),
 ('1182762--MethodDeclaration', '11827621606--matrixMultiplication'),
 ('1182762--MethodDeclaration', '118276289032--FormalParameter'),
 ('118276289032--FormalParameter', '11827628903299269--BasicType'),
 ('11827628903299269--BasicType', '1182762890329926926711--int'),
 ('118276289032--FormalParameter', '11827628903272534--matrix1'),
 ('1182762--MethodDeclaration', '118276263122--FormalParameter'),
 ('118276263122--FormalParameter', '11827626312223152--BasicType'),
 ('118276263122231

In [ ]:
#Removing the comments inside the file using regex
def rem_newline(item):
    item = item.replace('\n',"\\n")
    item = item.replace('\t',"\\t")
    return item

#Writing the nodes into a text file

In [ ]:
with open('java_nodes.txt', 'w') as fp:
    for item in nodes:
        # write each item on a new line
        item0=rem_newline(item[0])
        item1=rem_newline(item[1])
        #print(item0,item1)
        fp.write(str(item0)+","+str(item1)+"\n")
    print('Done')

0--CompilationUnit 11--ClassDeclaration
11--ClassDeclaration 1163223--set
1163223--set 116322317311--public
11--ClassDeclaration 113848--ON3_8
11--ClassDeclaration 1182762--MethodDeclaration
1182762--MethodDeclaration 118276243846--set
118276243846--set 11827624384630314--public
118276243846--set 11827624384619111--static
1182762--MethodDeclaration 118276229915--BasicType
118276229915--BasicType 11827622991598599--int
1182762--MethodDeclaration 11827621606--matrixMultiplication
1182762--MethodDeclaration 118276289032--FormalParameter
118276289032--FormalParameter 11827628903299269--BasicType
11827628903299269--BasicType 1182762890329926926711--int
118276289032--FormalParameter 11827628903272534--matrix1
1182762--MethodDeclaration 118276263122--FormalParameter
118276263122--FormalParameter 11827626312223152--BasicType
11827626312223152--BasicType 1182762631222315271599--int
118276263122--FormalParameter 11827626312279517--rows1
1182762--MethodDeclaration 118276265640--FormalParameter
11